In [2]:
import pyspark
from pyspark.sql import SparkSession

In [3]:
spark = SparkSession.builder \
        .master('local[*]') \
        .appName('test') \
        .getOrCreate()

23/03/01 22:35:04 WARN Utils: Your hostname, Bronx.local resolves to a loopback address: 127.0.0.1; using 192.168.1.112 instead (on interface en0)
23/03/01 22:35:04 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/03/01 22:35:04 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [4]:
!wc -l fhvhv_tripdata_2021-01.csv

 11908469 fhvhv_tripdata_2021-01.csv


In [11]:
df = spark.read \
      .option("header", "true") \
      .csv("fhvhv_tripdata_2021-01.csv")

In [6]:
df.show()

+-----------------+--------------------+-------------------+-------------------+------------+------------+-------+
|hvfhs_license_num|dispatching_base_num|    pickup_datetime|   dropoff_datetime|PULocationID|DOLocationID|SR_Flag|
+-----------------+--------------------+-------------------+-------------------+------------+------------+-------+
|           HV0003|              B02682|2021-01-01 00:33:44|2021-01-01 00:49:07|         230|         166|   null|
|           HV0003|              B02682|2021-01-01 00:55:19|2021-01-01 01:18:21|         152|         167|   null|
|           HV0003|              B02764|2021-01-01 00:23:56|2021-01-01 00:38:05|         233|         142|   null|
|           HV0003|              B02764|2021-01-01 00:42:51|2021-01-01 00:45:50|         142|         143|   null|
|           HV0003|              B02764|2021-01-01 00:48:14|2021-01-01 01:08:42|         143|          78|   null|
|           HV0005|              B02510|2021-01-01 00:06:59|2021-01-01 00:43:01|

In [7]:
df.head()

Row(hvfhs_license_num='HV0003', dispatching_base_num='B02682', pickup_datetime='2021-01-01 00:33:44', dropoff_datetime='2021-01-01 00:49:07', PULocationID='230', DOLocationID='166', SR_Flag=None)

In [8]:
# save the first 101 rows to head.csv, because pandas might be slow to read big data
!head -n 101 fhvhv_tripdata_2021-01.csv > head.csv

In [9]:
!head -n 10 head.csv

In [10]:
!wc -l head.csv

     101 head.csv


In [11]:
import pandas as pd

In [12]:
df_pandas = pd.read_csv('head.csv')

In [13]:
df_pandas.dtypes

hvfhs_license_num        object
dispatching_base_num     object
pickup_datetime          object
dropoff_datetime         object
PULocationID              int64
DOLocationID              int64
SR_Flag                 float64
dtype: object

In [12]:
spark.createDataFrame(df_pandas).schema

NameError: name 'df_pandas' is not defined

In [13]:
from pyspark.sql import types

In [14]:
schema = types.StructType([
    types.StructField('hvfhs_license_num', types.StringType(), True), 
    types.StructField('dispatching_base_num', types.StringType(), True), 
    types.StructField('pickup_datetime', types.TimestampType(), True), 
    types.StructField('dropoff_datetime', types.TimestampType(), True), 
    types.StructField('PULocationID', types.IntegerType(), True), 
    types.StructField('DOLocationID', types.IntegerType(), True), 
    types.StructField('SR_Flag', types.StringType(), True)
    ])

In [15]:
df = spark.read \
     .option('header', 'true') \
     .schema(schema) \
     .csv('fhvhv_tripdata_2021-01.csv') 

In [18]:
df.head(10)

[Row(hvfhs_license_num='HV0003', dispatching_base_num='B02682', pickup_datetime=datetime.datetime(2021, 1, 1, 0, 33, 44), dropoff_datetime=datetime.datetime(2021, 1, 1, 0, 49, 7), PULocationID=230, DOLocationID=166, SR_Flag=None),
 Row(hvfhs_license_num='HV0003', dispatching_base_num='B02682', pickup_datetime=datetime.datetime(2021, 1, 1, 0, 55, 19), dropoff_datetime=datetime.datetime(2021, 1, 1, 1, 18, 21), PULocationID=152, DOLocationID=167, SR_Flag=None),
 Row(hvfhs_license_num='HV0003', dispatching_base_num='B02764', pickup_datetime=datetime.datetime(2021, 1, 1, 0, 23, 56), dropoff_datetime=datetime.datetime(2021, 1, 1, 0, 38, 5), PULocationID=233, DOLocationID=142, SR_Flag=None),
 Row(hvfhs_license_num='HV0003', dispatching_base_num='B02764', pickup_datetime=datetime.datetime(2021, 1, 1, 0, 42, 51), dropoff_datetime=datetime.datetime(2021, 1, 1, 0, 45, 50), PULocationID=142, DOLocationID=143, SR_Flag=None),
 Row(hvfhs_license_num='HV0003', dispatching_base_num='B02764', pickup_dat

In [16]:
df = df.repartition(24)

In [17]:
df.write.parquet('fhv/2021/01')

23/03/01 22:38:40 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers


23/03/01 22:38:42 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers


23/03/01 22:38:43 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers


In [6]:
spark.sparkContext.uiWebUrl

'http://192.168.1.112:4040'

In [18]:
df = spark.read.parquet('fhv/2021/01')

In [19]:
df

DataFrame[hvfhs_license_num: string, dispatching_base_num: string, pickup_datetime: timestamp, dropoff_datetime: timestamp, PULocationID: int, DOLocationID: int, SR_Flag: string]

In [20]:
df.schema

StructType([StructField('hvfhs_license_num', StringType(), True), StructField('dispatching_base_num', StringType(), True), StructField('pickup_datetime', TimestampType(), True), StructField('dropoff_datetime', TimestampType(), True), StructField('PULocationID', IntegerType(), True), StructField('DOLocationID', IntegerType(), True), StructField('SR_Flag', StringType(), True)])

In [21]:
df.printSchema()

root
 |-- hvfhs_license_num: string (nullable = true)
 |-- dispatching_base_num: string (nullable = true)
 |-- pickup_datetime: timestamp (nullable = true)
 |-- dropoff_datetime: timestamp (nullable = true)
 |-- PULocationID: integer (nullable = true)
 |-- DOLocationID: integer (nullable = true)
 |-- SR_Flag: string (nullable = true)



In [23]:
df.select('pickup_datetime', 'pickup_datetime', 'pickup_datetime', 'pickup_datetime') \
  .filter(df.hvfhs_license_num == 'HV0003').show()

+-------------------+-------------------+-------------------+-------------------+
|    pickup_datetime|    pickup_datetime|    pickup_datetime|    pickup_datetime|
+-------------------+-------------------+-------------------+-------------------+
|2021-01-01 16:17:16|2021-01-01 16:17:16|2021-01-01 16:17:16|2021-01-01 16:17:16|
|2021-01-04 15:44:57|2021-01-04 15:44:57|2021-01-04 15:44:57|2021-01-04 15:44:57|
|2021-01-03 14:05:52|2021-01-03 14:05:52|2021-01-03 14:05:52|2021-01-03 14:05:52|
|2021-01-01 09:27:50|2021-01-01 09:27:50|2021-01-01 09:27:50|2021-01-01 09:27:50|
|2021-01-04 10:53:41|2021-01-04 10:53:41|2021-01-04 10:53:41|2021-01-04 10:53:41|
|2021-01-04 22:29:41|2021-01-04 22:29:41|2021-01-04 22:29:41|2021-01-04 22:29:41|
|2021-01-02 14:52:09|2021-01-02 14:52:09|2021-01-02 14:52:09|2021-01-02 14:52:09|
|2021-01-01 07:31:49|2021-01-01 07:31:49|2021-01-01 07:31:49|2021-01-01 07:31:49|
|2021-01-02 22:35:51|2021-01-02 22:35:51|2021-01-02 22:35:51|2021-01-02 22:35:51|
|2021-01-01 15:3

In [24]:
from pyspark.sql import functions as F

In [29]:
df.show()

+-----------------+--------------------+-------------------+-------------------+------------+------------+-------+
|hvfhs_license_num|dispatching_base_num|    pickup_datetime|   dropoff_datetime|PULocationID|DOLocationID|SR_Flag|
+-----------------+--------------------+-------------------+-------------------+------------+------------+-------+
|           HV0003|              B02884|2021-01-01 16:17:16|2021-01-01 16:35:20|         153|         167|   null|
|           HV0003|              B02882|2021-01-04 15:44:57|2021-01-04 15:56:21|         127|         244|   null|
|           HV0005|              B02510|2021-01-04 14:29:31|2021-01-04 14:54:56|         138|          49|   null|
|           HV0003|              B02864|2021-01-03 14:05:52|2021-01-03 14:20:29|         183|          20|   null|
|           HV0003|              B02867|2021-01-01 09:27:50|2021-01-01 09:46:50|         162|         106|   null|
|           HV0005|              B02510|2021-01-05 11:26:21|2021-01-05 11:55:13|

In [32]:
# a function that does crazy stuff for some weird business requirement
# can't use sql
def crazy_stuff(base_num):
    num = int(base_num[1:])
    if num % 7 == 0:
        return f's/{num:03x}'
    else:
        return f'e/{num:03x}'
# a sample why spark is better than just having sql, because we can use functions that we can test

In [33]:
crazy_stuff('B02877')

's/b3d'

In [36]:
# now sample udf 
# udf is user define function
crazy_stuff_udf = F.udf(crazy_stuff, returnType=types.StringType())

In [38]:
df \
    .withColumn('pickup_date', F.to_date(df.pickup_datetime)) \
    .withColumn('dropoff_date', F.to_date(df.dropoff_datetime)) \
    .withColumn('base_id', crazy_stuff_udf(df.dispatching_base_num)) \
    .select('pickup_date', 'dropoff_date', 'base_id', 'PULocationID', 'DOLocationID') \
    .show()

+-----------+------------+-------+------------+------------+
|pickup_date|dropoff_date|base_id|PULocationID|DOLocationID|
+-----------+------------+-------+------------+------------+
| 2021-01-01|  2021-01-01|  s/b44|         153|         167|
| 2021-01-04|  2021-01-04|  e/b42|         127|         244|
| 2021-01-04|  2021-01-04|  e/9ce|         138|          49|
| 2021-01-03|  2021-01-03|  e/b30|         183|          20|
| 2021-01-01|  2021-01-01|  e/b33|         162|         106|
| 2021-01-05|  2021-01-05|  e/9ce|         216|         265|
| 2021-01-04|  2021-01-04|  e/b48|          18|          18|
| 2021-01-05|  2021-01-05|  e/9ce|          42|         207|
| 2021-01-04|  2021-01-04|  e/b3b|          76|         215|
| 2021-01-02|  2021-01-02|  e/acc|         220|          32|
| 2021-01-01|  2021-01-01|  e/b3b|         167|          94|
| 2021-01-02|  2021-01-02|  e/b32|          71|         130|
| 2021-01-01|  2021-01-01|  e/b38|         222|          61|
| 2021-01-03|  2021-01-0

In [28]:
df.select('pickup_datetime', 'pickup_datetime', 'pickup_datetime', 'pickup_datetime') \
  .filter(df.hvfhs_license_num == 'HV0003').show()

+-------------------+-------------------+-------------------+-------------------+
|    pickup_datetime|    pickup_datetime|    pickup_datetime|    pickup_datetime|
+-------------------+-------------------+-------------------+-------------------+
|2021-01-01 16:17:16|2021-01-01 16:17:16|2021-01-01 16:17:16|2021-01-01 16:17:16|
|2021-01-04 15:44:57|2021-01-04 15:44:57|2021-01-04 15:44:57|2021-01-04 15:44:57|
|2021-01-03 14:05:52|2021-01-03 14:05:52|2021-01-03 14:05:52|2021-01-03 14:05:52|
|2021-01-01 09:27:50|2021-01-01 09:27:50|2021-01-01 09:27:50|2021-01-01 09:27:50|
|2021-01-04 10:53:41|2021-01-04 10:53:41|2021-01-04 10:53:41|2021-01-04 10:53:41|
|2021-01-04 22:29:41|2021-01-04 22:29:41|2021-01-04 22:29:41|2021-01-04 22:29:41|
|2021-01-02 14:52:09|2021-01-02 14:52:09|2021-01-02 14:52:09|2021-01-02 14:52:09|
|2021-01-01 07:31:49|2021-01-01 07:31:49|2021-01-01 07:31:49|2021-01-01 07:31:49|
|2021-01-02 22:35:51|2021-01-02 22:35:51|2021-01-02 22:35:51|2021-01-02 22:35:51|
|2021-01-01 15:3